## Goal: What is Fst score per exon + ss (+-2bp)
For exons that are 
* unique to one population or the other
* shared between the population

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [3]:
df = pd.read_csv(proc_cfg(config['lr']['exons_ss_in_pops_fsts_summary'], od),
                 sep='\t')
df.rename({'ThickStart':'eid',
           'ThickEnd':'novelty',
           'ItemRGB':'pop1_det',
           'BlockCount':'pop2_det'}, 
          axis=1, inplace=True)

# set all <0 fsts --> 0 https://www.biostars.org/p/132253/
df.loc[df.fst<0, 'fst'] = 0

In [4]:
gb_cols = ['eid', 'novelty', 'pop1_det', 'pop2_det',
            'pop1', 'pop2']
keep_cols = gb_cols+['fst']


# make sure there are no dupe eid + contrast + var. posiition
# assert len(df[gb_cols+['End_b', 'Start_b']].drop_duplicates().index) == len(df.index) # oop there's a bug somewhere upstream w/ the vcf processing

print('REMOVE ME WHEN FABIEN FIXES FST STUFF')
print(len(df.index))
df = df.drop_duplicates(subset=gb_cols+['End_b', 'Start_b'], keep=False)
print(len(df.index))

# add up each fst per contrast and exon
df = df.groupby(gb_cols).sum().reset_index().rename({'fst':'fst_sum'}, axis=1)
df.head()

REMOVE ME WHEN FABIEN FIXES FST STUFF
1459932
1458930


,eid,novelty,pop1_det,pop2_det,pop1,pop2,Start,End,End_b,fst_sum,Start_b
0,GL000008.2_+_135133_135173,Known,False,True,CEU,PEL,135133,135173,-1,0.0,-1
1,GL000008.2_+_135133_135173,Known,False,True,ITU,PEL,135133,135173,-1,0.0,-1
2,GL000008.2_+_135133_135173,Known,False,True,LWK,PEL,135133,135173,-1,0.0,-1
3,GL000008.2_+_135133_135173,Known,True,False,PEL,YRI,135133,135173,-1,0.0,-1
4,GL000008.2_+_155429_155531,Known,False,True,CEU,ITU,155429,155531,-1,0.0,-1


In [5]:
# df.loc[df[gb_cols+['End_b', 'Start_b']].duplicated(keep=False)].sorts_values(by=gb_cols+['End_b', 'Start_b'])

In [7]:
# get length of each exon
df['elen'] = df['End']-df['Start']

# and get avg. Fst per base in each exon+ss
df['avg_fst_per_base'] = df['fst_sum']/df['elen']

In [8]:
df.head()

,eid,novelty,pop1_det,pop2_det,pop1,pop2,Start,End,End_b,fst_sum,Start_b,elen,avg_fst_per_base
0,GL000008.2_+_135133_135173,Known,False,True,CEU,PEL,135133,135173,-1,0.0,-1,40,0.0
1,GL000008.2_+_135133_135173,Known,False,True,ITU,PEL,135133,135173,-1,0.0,-1,40,0.0
2,GL000008.2_+_135133_135173,Known,False,True,LWK,PEL,135133,135173,-1,0.0,-1,40,0.0
3,GL000008.2_+_135133_135173,Known,True,False,PEL,YRI,135133,135173,-1,0.0,-1,40,0.0
4,GL000008.2_+_155429_155531,Known,False,True,CEU,ITU,155429,155531,-1,0.0,-1,102,0.0
